In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as st
from scipy.stats import linregress

# 1. Data Retrieval and Cleanup

In this section, we will prepare our raw CDC data for visual and quantitative analysis. We were very fortunate to find such a wealth of data. Thank you, CDC! Only about 6% of the raw dataset could be considered "clean" (no missing datapoints), but the original dataset was so large that we retained nearly half a million rows of patient data.

In [ ]:
df = pd.read_csv('http://data.cdc.gov/api/views/vbim-akqf/rows.csv?accessType=DOWNLOAD')
# If you already have the csv file downloaded, replace the above code with: 
    # df = pd.read_csv(<PATH_TO_CSV>)


In [ ]:
#Rename columns for ease of visibility
renames = {'Black, Non-Hispanic': 'Black', 'White, Non-Hispanic': 'White', 'Hispanic/Latino': 'Hispanic',
           'Asian, Non-Hispanic': 'Asian', 'Multiple/Other, Non-Hispanic':'Other', 
           'American Indian/Alaska Native, Non-Hispanic': 'Native', 
           'Native Hawaiian/Other Pacific Islander, Non-Hispanic': 'Pacific Islander'}
for key in renames.keys():
    df = df.replace(key, renames[key])
df.head()

In [ ]:
# Extract month from date
#df.to_csv('new.csv')
def getMonth(date):
    try:
        return date[5:7]
    except:
        return date
df['month'] = df['cdc_report_dt'].apply(getMonth)


In [ ]:
# Get rid of missing and unknown data
df = df.replace(['Missing', 'Unknown'], np.nan) #no interest in these rows
df = df.replace('No', 0)
df = df.replace('Yes', 1)

df = df.replace('Male', 1)
df = df.replace('Female', 0)

df['sex'] = df['sex'].replace('Other', np.nan) #Negligle - only like 30 people are "other" in this dataset

#We don't mind keeping nan values for date data, 
#so we need to replace nan with something that won't be dropped by dropna
df['pos_spec_dt'] = df['pos_spec_dt'].replace(np.nan, '--')
df['onset_dt'] = df['onset_dt'].replace(np.nan, '--')
dropped = df.dropna()
df.head()

# We still have almost half a million rows that have every single one of our desired datapoints

In [ ]:
# Save clean dataframe
dropped.to_csv('Data/clean_data.csv', index=False)

## Bias
### Have we introduced bias into our sample?
We have cut down our sample size by more than 90%. Thankfully, even when we're extremely picky, and we don't allow for *any* datapoints to be missing (convenient for quantitative analysis), we still have almost half a *million* cases of Covid-19 to examine.

Thanks to the CDC's huge amount of data collected, we can prune down the data, and we still have 10x the sample size that Pfizer had for its phase 3 Covid vaccine trials (which just got approved

So, we don't have to worry about whether or not we have removed *too much* of the dataset, but we do have to worry about the possibility of bias. Did we remove rows randomly, or did we introduce bias? 

---
### Random selection
To understand why the question of randomness is so important, think of a simple example: You have a dataset of one billion men, sampled from the globe at random. The two columns of information are height and nationality. You find that the average height for men is 176.5 cm. Now, think of three distinct scenarios:

Scenario 1: You RANDOMLY select 1000 (0.1%) of the men. Do you think the average height will still be 176.5 cm?

Scenario 2: You select 1000 men, but you make sure they are ALL from the country of Norway (an especially tall country). Now, do you think the new average will reflect the global average?

Scenario 3: You select 1000 men, but you make sure that you ONLY select men who wore red hats today. Now, do you think the new average will reflect the global average?

Both scenario 2 and 3 are highly UN-random, but only one is a biased sample of height. The sample average height for scenario 3 should yield the same value as the global population average. This is because nationality correlates with height, and the wearing of red hats does not.

---
### CDC Data Sampling
This is the question we need to be asking ourselves: Is our selection criteria (the full availability of hospital data) correlated with our dependent variable (death)?
When we drop these 7.5 million incomplete rows, is it more akin to the nationality example, or the red hats example?

We can explore this question intuitively and quantitatively. Intuitively, I would make the argument that there is unlikely to be any correlation between a hospital's data-reporting and its patient outcomes. I imagine that the staff who generate these datasets are not doctors, and the quality of their work does not reflect the quality of the medical work at these hospitals. I would assume based on intuition that the quality of medical care and the quality of reported data are completely independent of one another. 

One great way to look for bias is to visualize the distribution of population fields. In this case, intuition prevails. The distributions are nearly identical after performing the data pruning.

In [ ]:
#Generate a histogram of the two dataframes - one with dropped values, the other without
def compareHistogram(col):
    fig, axs = plt.subplots(1,2, figsize= (20,8))
    fig.suptitle(col)
    
    df[col].hist(ax = axs[0])
    axs[0].set_title('Before Dropping:')

    dropped[col].hist(ax = axs[1])
    axs[1].set_title('After Dropping')
    plt.savefig('Images/drop_data_comparisons/' + col + '.png', facecolor = 'white')

    
for column in ['month','race_ethnicity_combined', 'age_group', 'hosp_yn','icu_yn','death_yn','medcond_yn', 'sex']:
    compareHistogram(column)

# 1.1 Gather Hospital Data from Health.gov 
We will group this data by month so as to get a general picture of the country 

State-by-state information (like the information in this particular dataset) is not of any interest to our analysis, because there is no location data in the CDC dataset

In [ ]:
df = pd.read_csv('https://healthdata.gov/sites/default/files/reported_hospital_utilization_timeseries_20201213_2141.csv')
df

In [ ]:
occupancy = df[['date', 'adult_icu_bed_covid_utilization', 'inpatient_beds_utilization', 'percent_of_inpatients_with_covid']]

def getMonth(date):
    return date[5:7]

# Group by month
occupancy['month'] = occupancy['date'].apply(getMonth)
occupancy = occupancy.groupby('month', dropna = False).mean()[['adult_icu_bed_covid_utilization', 'inpatient_beds_utilization', 'percent_of_inpatients_with_covid']]
occupancy

In [ ]:
# Save Data
df.to_csv('Data/hospital_info.csv', index=False)
occupancy.to_csv('Data/hospital_occupancy.csv')

# 1.2 Merge Hospital Data into Main CDC Dataset

In [ ]:
df = pd.read_csv('Data/clean_data.csv')
cap = pd.read_csv('Data/hospital_occupancy.csv', index_col = False)
cap

In [ ]:
# Perform Merge
df = pd.merge(df, cap[['month', 'inpatient_beds_utilization', 'percent_of_inpatients_with_covid']], on = 'month').rename(columns={
    
    'inpatient_beds_utilization': 'inpatient__occupancy'
    
})
df.head()

In [ ]:
# Save (and overwrite) data.
df.to_csv('Data/clean_data.csv', index= False)

# 2. Data Exploration and Visualization

## Gender

## Are more men dying from COVID-19 than women?

In [ ]:
df = pd.read_csv('./Data/clean_data.csv')

# Are more men dying from COVID-19 than women?

clean_data_df = df

# Calculate the distribution of female versus male COVID-19 Cases
total_cases= len(clean_data_df['sex'])

by_gender_count=clean_data_df["sex"].value_counts()

by_gender_pct = (100*(by_gender_count/total_cases)).round(2)

# Calculate the distribution of female versus male deaths by COVID-19
death = clean_data_df[(clean_data_df['death_yn'] == 1.0)]
total_death = len(death['death_yn'])
by_gender_death=death["sex"].value_counts()
by_gender_death_pct = (100*(by_gender_death/total_death)).round(2)


# Calculate the distribution of female versus male deaths with underlying medical condition by COVID-19
death_med_cond = death[(death['medcond_yn'] == 1.0)]
total_death_med_cond = len(death_med_cond['medcond_yn'])
by_gender_death_med_cond = death_med_cond["sex"].value_counts()
by_gender_death_med_cond_pct = (100*(by_gender_death_med_cond/total_death_med_cond)).round(2)

# Assemble the resulting series into a single summary dataframe.
gender_df = pd.DataFrame({"Sex": ["Female", "Male"],
                          "Number of Cases": by_gender_count,
                          "Share of Cases (%)": by_gender_pct,
                          "Number of Deaths":by_gender_death,
                          "Share of Deaths (%)":by_gender_death_pct,
                          "Deaths with underlying condition": by_gender_death_med_cond,
                          "Share of Death with underlying condition (%)":by_gender_death_med_cond_pct })

gender_df

In [ ]:
### COVID-19 Cases and Deaths Disaggregated by Sex

# Pie plot showing the distribution of male versus female COVID-19 Cases

# Create Labels for the sections of the pie
labels = gender_df["Sex"]
distribution = gender_df["Share of Cases (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['green', 'blue'],explode = (0.1, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Cases by Sex")

plt.savefig('Images/US_Covid_cases_by_sex.png')
plt.show()

# Pie plot showing the distribution of male versus female deaths by COVID-19 Cases

# Create Labels for the sections of the pie
labels = gender_df["Sex"]
distribution = gender_df["Share of Deaths (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['green', 'blue'],explode = (0.1, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Deaths by Sex")

plt.savefig('Images/US_Covid_deaths_by_sex.png')
plt.show()

After analyzing and comparing the pie chart U.S. COVID-19 cases and deaths, it is possible to observe that the female gender has a larger percentage of getting the COVID disease and the male gender has a larger percentage of deaths. The ratio of COVID-19 by sex mortality rate shows that men has 1.4 chances of dying compared to women indicating that for every 10 deaths among confirmed cases in women, there are 14 death in men.

In [ ]:
### Rate Ratio of COVID-19 by Sex Mortality Rate

total_pct_deaths =(100*(by_gender_death/total_cases)).round(2)


sex_ratio_df = pd.DataFrame({"Sex": ["Female", "Male"],
                          "Number of Cases": by_gender_count,
                          "Number of Deaths":by_gender_death,
                          "Total Deaths (%)":total_pct_deaths})
sex_ratio_df

In [ ]:
male_female_radio = (total_pct_deaths[1]/total_pct_deaths[0]).round(1)
print(f'Men have {male_female_radio} chances of dying than women.')
print(f'For every 10 deaths among confirmed cases in women, there are {"{:.0f}".format(10*male_female_radio)} death in men.')

### Possible Risk Factors  

# Pie plot showing the distribution of male versus female deaths by COVID-19 Cases with underlying medical conditions

# Create Labels for the sections of the pie
labels = gender_df["Sex"]
distribution = gender_df["Share of Death with underlying condition (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['green', 'blue'],explode = (0.1, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Deaths with Underlying Medical Condition by Sex")

plt.savefig('Images/US_Covid_deaths_with_medcond_by_sex.png')
plt.show()

A pie plot of deaths related to people with underlying medical conditions was generated to understand the possible risk factor of why more men are dying due to COVID-19. The result shows that the percentage of female and male remains the same as the pie plot “U.S. Covid-19 deaths by sex” shown previously indicating that underlying condition does not change the odds of men dying from COVID-19 over women.

In [ ]:
# Calculating the distribution of male versus female deaths by COVID-19 Cases base on age groups

# Number of death in womens by age groups
woman_death = clean_data_df[((clean_data_df['sex'] == 0) & (clean_data_df['death_yn'] == 1.0))]
woman_age_group_deaths = woman_death.groupby(["age_group"]).count()["death_yn"]

# Number of death in male by age group 
men_death = clean_data_df[((clean_data_df['sex'] == 1.0) & (clean_data_df['death_yn'] == 1.0))]
men_age_group_deaths = men_death.groupby(["age_group"]).count()["death_yn"]
men_age_group_deaths

# Bar plot showing the distribution of male versus female deaths by COVID-19 Cases base on age groups

labels = ['0 - 9 Years', '10 - 19 Years', '20 - 29 Years', '30 - 39 Years',
        '40 - 49 Years', '50 - 59 Years', '60 - 69 Years', '70 - 79 Years', '80+ Years']
y = woman_age_group_deaths
y1 = men_age_group_deaths

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (15,8))
rects1 = ax.bar(x - width/2, y, width, label='Female', color='g')
rects2 = ax.bar(x + width/2, y1, width, label='Male', color='b')
ax.set_xlabel('Age Group')
ax.set_ylabel('Number of Deaths')
ax.set_title('U.S. COVID-19 Deaths by Sex and Age Group')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation='vertical')
ax.legend()
fig.tight_layout()

plt.savefig('Images/US_Covid_deaths_by_sex_and_age.png')
plt.show()

Another factor taken into consideration in this analysis was the age group associated with U.S Covid-19 deaths. The bar plot shows the distribution of number of deaths in males and females vs the age groups. The result shows a consistent higher percentage of male dying compared to women (excluding age 80+ age group) and it is more pronounced between males of age groups 60 to 79 years. In the 80+ years group, the relationship changes (more females dying over male) but research studies from other countries show a consistent higher percentage of male dying compared to women in all age groups. This bias in the data is related to women living six years longer than men on average, meaning that there are more elderly women in the vulnerable population than men.

### Are people with underlying health conditions at a higher risk of having serious symptoms of COVID-19?

In [ ]:
## Are people with underlying health conditions at higher risk of serious symptoms of COVID-19?

clean_data_df = df

# Calculate the distribution of COVID-19 Cases with underlying health conditions.
total_cases= len(clean_data_df['medcond_yn'])
med_cond_count=clean_data_df["medcond_yn"].value_counts()
med_cond_pct = (100*(med_cond_count/total_cases)).round(2)

# Calculate the distribution with underlying health conditions deaths by COVID-19
death = clean_data_df[(clean_data_df['death_yn'] == 1.0)]
total_death = len(death['death_yn'])
med_cond_death=death["medcond_yn"].value_counts()
med_cond_death_share = (100*(med_cond_death/total_death)).round(2)
med_cond_death_pct = (100*(med_cond_death/total_cases)).round(2)

# Calculate the distribution hospitalization with underlying medical conditions by COVID-19
hospitalization = clean_data_df[(clean_data_df['hosp_yn'] == 1.0)]
total_hospitalization = len(hospitalization['hosp_yn'])
med_cond_hospitalization=hospitalization["medcond_yn"].value_counts()
med_cond_hospitalization_share = (100*(med_cond_hospitalization/total_hospitalization)).round(2)
med_cond_hospitalization_pct = (100*(med_cond_hospitalization/total_cases)).round(2)

# Calculate the distribution ICU admission with underlying medical conditions by COVID-19
icu = clean_data_df[(clean_data_df['icu_yn'] == 1.0)]
total_icu = len(icu['icu_yn'])
med_cond_icu=icu["medcond_yn"].value_counts()
med_cond_icu_share = (100*(med_cond_icu/total_icu)).round(2)
med_cond_icu_pct = (100*(med_cond_icu/total_cases)).round(2)

# Assemble the resulting series into a single summary dataframe.
med_cond_df = pd.DataFrame({"Medical Condition": ["Yes", "No"],
                          "Number of Cases": med_cond_count,
                          "Cases (%)": med_cond_pct,
                          "Number of Hospitalizations": med_cond_hospitalization,
                          "Share of Hospitalizations (%)": med_cond_hospitalization_share,
                          "Hospitalizations (%)": med_cond_hospitalization_pct, 
                          "Number of ICU Admissions": med_cond_icu,
                          "Share of ICU Admissions (%)": med_cond_icu_share,
                          "ICU Admissions (%)": med_cond_icu_pct,
                          "Number of Deaths": med_cond_death,
                          "Share of Death (%)": med_cond_death_share,
                          "Death (%)": med_cond_death_pct})
med_cond_df

In [ ]:
### Plots of distribution of COVID-19 Cases by shares with and without underlying health conditions.

# Pie plot showing the distribution of COVID-19 Cases with and without underlying health conditions

# Create Labels for the sections of the pie
labels = ["With Underlying Medical Condition","Without Underlying Medical Condition" ]
distribution = med_cond_df["Cases (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['orange', 'blue'],explode = (0.1, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Cases with underlying health conditions")

plt.savefig('Images/US_Covid_cases_with_medcond.png')
plt.show()

# Pie plot showing the distribution of COVID-19 Hospitalizations with and without underlying health conditions

# Create Labels for the sections of the pie
labels = ["With Underlying Medical Condition","Without Underlying Medical Condition" ]
distribution = med_cond_df["Share of Hospitalizations (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['orange', 'blue'],explode = (0.05, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Hospitalizations with underlying health conditions")

plt.savefig('Images/US_Covid_hospitalizations_with_medcond.png')
plt.show()

In [ ]:
# Pie plot showing the distribution of COVID-19 ICU admissions with and without underlying health conditions

# Create Labels for the sections of the pie
labels = ["With Underlying Medical Condition","Without Underlying Medical Condition" ]
distribution = med_cond_df["Share of ICU Admissions (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['orange', 'blue'],explode = (0.05, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 ICU admissions with underlying health conditions")

plt.savefig('Images/US_Covid_ICU_with_medcond.png')
plt.show()

In [ ]:
# Pie plot showing the distribution of COVID-19 Deaths with and without underlying health conditions

# Create Labels for the sections of the pie
labels = ["With Underlying Medical Condition","Without Underlying Medical Condition" ]
distribution = med_cond_df["Share of Death (%)"]

# Pie plot using pyplot
plt.pie(distribution, labels=labels, colors = ['orange', 'blue'],explode = (0.05, 0), autopct="%1.1f%%",shadow=True, startangle=90 )

#Formatting
plt.title("U.S. COVID-19 Deaths with underlying health conditions")

plt.savefig('Images/US_Covid_deaths_with_medcond.png')
plt.show()

In [ ]:
### Bar plot of distribution of COVID-19 Risk with underlying health conditions.

# Bar plot showing the distribution of COVID-19 Cases with underlying health conditions.

labels = ['Underlying health conditions', 'No underlying health conditions']
y1 = med_cond_hospitalization_pct
y2 = med_cond_icu_pct
y3 = med_cond_death_pct

x = np.arange(len(labels))  # the label locations
width = 0.2


fig, ax = plt.subplots()
rects2 = ax.bar(x - width, y1, width, label='Hospitalization', color='b')
rects3 = ax.bar(x , y2, width, label='ICU admission', color='y')
rects4 = ax.bar(x + width, y3, width, label='Death', color='r')
ax.set_ylabel('% of group')
ax.set_title('U.S. COVID-19 Distribution of Cases with underlying health conditions')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation='horizontal')
ax.legend()

plt.savefig('Images/US_Covid_distribution_of_cases_with_medcond.png')
plt.show()

A bar plot was generated to show the distribution of percentages of hospitalizations, ICU admissions and death for people with and without underlying medical conditions. The objective of this plot is to further interrogate the data and try to understand if people with underlying health conditions are at higher risk of serious symptoms of COVID-19. The results clearly show a higher risk of getting hospitalized, admitted to the ICU unit or fatalities in people with underlying medical conditions.

In [ ]:
###  Risk Rate Ratio of COVID-19 Cases with underlying health conditions.

# Hospitalization risk ratio
hospitalization_radio = (med_cond_hospitalization_pct[1]/med_cond_hospitalization_pct[0]).round(1)
print(f'COVID-19 patients with an underlying medical condition are {hospitalization_radio} times as likely to be hospitalized.')

# ICU admission risk ratio
ICU_radio = (med_cond_icu_pct[1]/med_cond_icu_pct[0]).round(1)
print(f'COVID-19 patients with an underlying medical condition are {ICU_radio} times as likely to be ICU admitted.')

# Death risk ratio
death_radio = (med_cond_death_pct[1]/med_cond_death_pct[0]).round(1)
print(f'COVID-19 patients with an underlying medical condition are {ICU_radio} times as likely to die.')

In [ ]:
### Conditions contributing to deaths involving coronavirus disease 2019 (COVID-19) in U.S.

# Import the data from Centers for Disease Control and Prevention
med_condition_df = pd.read_csv('http://data.cdc.gov/api/views/hk9y-quqm/rows.csv?accessType=DOWNLOAD')

# Data conditioning
med_condition_df = med_condition_df.replace('COVID-19', 0)
indexNames = med_condition_df[med_condition_df['Condition Group'] == 0 ].index
med_condition_df.drop(indexNames , inplace=True)

# Conditions contributing to deaths involving coronavirus disease 2019 (COVID-19) in U.S. 
med_condition_df

In [ ]:
# Save clean dataframe
med_condition_df.to_csv('./Data/preexisten_med_conditions_data.csv', index=False)

# Medical Condition data frame
drop_columns = med_condition_df.drop(columns=['Data as of','Start Week', 'End Week','State', 'ICD10_codes', 'Age Group', 'Number of Mentions', 'Flag'])
drop_columns

condition_df = pd.DataFrame(drop_columns.groupby(["Condition Group"]).count())
condition_df.columns = ["Total Count", "Total Deaths"]
condition_df = condition_df.sort_values(by='Total Count', ascending=False)

# Calculate percentage
condition_df["Condition Percentage (%)"] = (100*(condition_df["Total Count"]/condition_df["Total Count"].sum())).round(2)

condition_df

In [ ]:
# Pie plot showing conditions contributing to deaths involving coronavirus disease 2019 (COVID-19) in U.S. 

# Create Labels for the sections of the pie
labels = condition_df.index
condition = condition_df['Condition Percentage (%)']
explode = (0.05, 0.05, 0, 0, 0, 0, 0, 0, 0, 0, 0)

# Pie plot using pyplot
fig1, ax1 = plt.subplots(figsize=(8, 8))
ax1.pie(condition, explode=explode, labels=labels , autopct='%1.1f%%', shadow=True)
ax1.axis('equal')  

#Formatting
plt.title("Conditions contributing to deaths involving coronavirus disease 2019 (COVID-19) in U.S.")

plt.savefig('Images/US_Covid_med_conditions_contributing_death.png')
plt.show()

The pie plot shows the primary health conditions contributing to deaths associated with COVID-19 disease in the U.S.

## Ethnicity

In [ ]:
#importing the csv file
file = pd.read_csv('./Data/clean_data.csv')
file.head()

In [ ]:
#organizing different enthnicities by month

dates = file[['month','race_ethnicity_combined',"death_yn"]]

total_cases_by_month = dates.groupby(['month','race_ethnicity_combined']).count()
total_cases_by_month

In [ ]:
#pulling out just the deaths
deaths = dates.groupby(dates["month"]).sum()
deaths

In [ ]:
#getting the deaths of different races
ethnicity = dates.groupby([dates['month'],dates['race_ethnicity_combined']]).sum()
#getting the total cases of different races
ethnic_story = dates.groupby([dates['month'],dates['race_ethnicity_combined']]).count()
ethnic_story = ethnic_story.unstack()
ethnicity = ethnicity.unstack()
ethnicity

In [ ]:
#organizing data into pie charts
pie_chart = file.groupby("race_ethnicity_combined")["death_yn"].sum()
cases = file.groupby("race_ethnicity_combined")["death_yn"].count()
#pie_chart
cases

In [ ]:
#plotting the pie chart
plot = pie_chart.plot.pie(y=pie_chart[1], autopct='%.2f',title="Covid Deaths of Different Races")

In [ ]:
#displaying the percentage of total cases by ethnicity
plot = cases.plot.pie(y=pie_chart[1], autopct="%.2f", title="Total Cases of Different Ethnicities")

In [ ]:
#building a bar graph of the cases and deaths to compare percentages
bar_df = pd.DataFrame(pie_chart)
bar_df['total cases'] = cases
bar_df

In [ ]:
#displaying the bar graph
bar_graph = bar_df.plot.bar(figsize=(15,8))

In [ ]:
#survival rate of cases vs deaths for Asian people
pie_chart_asian = [bar_df['death_yn']['Asian'],
                   (bar_df['total cases']['Asian']-bar_df['death_yn']['Asian'])]
pie_chart_asian

plt.title("Deaths of Asians")
chart = plt.pie(pie_chart_asian,autopct='%.2f')

In [ ]:
#survival rate of Black people
pie_chart_black = [bar_df['death_yn']['Black'],
                   (bar_df['total cases']['Black']-bar_df['death_yn']['Black'])]
pie_chart_black
plt.title("Deaths of Black People")
chart = plt.pie(pie_chart_black,autopct='%.2f')

In [ ]:
#survival rate of Hispanic people
pie_chart_hisp = [bar_df['death_yn']['Hispanic'],
                   (bar_df['total cases']['Hispanic']-bar_df['death_yn']['Hispanic'])]
pie_chart_hisp
plt.title('Deaths of Hispanic People')
chart = plt.pie(pie_chart_hisp,autopct='%.2f')

In [ ]:
#survival rate of Native American people
pie_chart_native = [bar_df['death_yn']['Native'],
                   (bar_df['total cases']['Native']-bar_df['death_yn']['Native'])]
pie_chart_native
plt.title('Deaths of Native Americans')
chart = plt.pie(pie_chart_native,autopct='%.2f')

In [ ]:
#survival rate of the group of people of other ethnicity
pie_chart_other = [bar_df['death_yn']['Other'],
                   (bar_df['total cases']['Other']-bar_df['death_yn']['Other'])]
pie_chart_other
plt.title("Deaths of Other Races")
chart = plt.pie(pie_chart_other,autopct='%.2f')

In [ ]:
#suvival rate of Pacific Islander people
pie_chart_pac = [bar_df['death_yn']['Pacific Islander'],
                   (bar_df['total cases']['Pacific Islander']-bar_df['death_yn']['Pacific Islander'])]
pie_chart_pac
plt.title("Deaths of Pacific Islanders")
chart = plt.pie(pie_chart_pac,autopct='%.2f')

In [ ]:
#survival rate of White people
pie_chart_white = [bar_df['death_yn']['White'],
                   (bar_df['total cases']['White']-bar_df['death_yn']['White'])]
pie_chart_white
plt.title("Deaths of White People")
chart = plt.pie(pie_chart_white,autopct='%.2f')

### Going through all the pie charts there are some interesting notes to take.
The deaths of Pacific Islanders in our data set are only 4.19% of the cases of Pacific Islanders.
White people, Hispanic people, and other races have a ~5-7% death rate in this data set, but native Americans and black people have a relatively high death rate of around 10%. This could be due to access to proper healthcare, but that wouldn't explain why hispanic people die less often than white people when white people more frequently have health insurance(1).
Perhaps hispanic people have different a different reaction to Covid-19 than other races, or perhaps culturally they treat their sick ones differently than other races.

1: https://www.statista.com/statistics/200970/percentage-of-americans-without-health-insurance-by-race-ethnicity/

In [ ]:
#building a dataframe of months and different races deaths
#which I already did in my other file
time=file[['race_ethnicity_combined','month','death_yn']]

In [ ]:
#putting together a dataframe of 
#information from the census and my findings
population_info = {"Ethnicity":['White','Hispanic','Black','Asian','Native American','Pacific Islander','Other'],
                  'Total Population':[200226109,60724311,43984096,12366132,4267114,656479,1969437],
                  'Percentage of US Population':[60.1,18.5,13.4,5.9,1.3,0.2,0.6],
                  'Percentage of Total Cases':[59.65,21.35,12.15,2.95,0.36,0.56,np.nan],
                  'Percentage of Total Deaths':[55.82,16.01,20.03,4.29,0.51,0.36,np.nan],
                  'Percentage of Ethnicity Reporting Covid':[0.136,0.16,0.126,0.0697,0.0385,0.062,np.nan]}
population_df = pd.DataFrame(population_info)
population_df

# Percentages of populations
In this chart I wanted to collate some data I found both in my research and from the census. Since I noticed that hispanic people tend to survive more often I wanted to actually dig into the numbers a little bit. Here you can see that while hispanic people only account for 18.5% of the total US population, they had 21.35% of the cases of covid in our data set. Interestingly, they only have 16.01% of the deaths in our data set. Which is a lower percentage than that of their respective part of the total US population. So what we're seeing here is that hispanic people are suviving more than other races, despite being less insured.

Interestingly, Asian people have presumably been much more responsible about social distancing, as their percentage of the total cases is about half of their percentage of the total US population. 

Black people have been hit the hardest, with 20% of the total deaths from covid, while only making up 13.4% of the total population.

1: https://www.census.gov/quickfacts/fact/table/US/RHI725219

In [ ]:
#building new dataframe for gender and race
genders = file[['sex','race_ethnicity_combined',"death_yn"]]
#getting the deaths of different races
ethnicity_gender = genders.groupby([genders['sex'],genders['race_ethnicity_combined']]).sum()
#getting the total cases of different races
ethnic_gender = genders.groupby([genders['sex'],genders['race_ethnicity_combined']]).count()
ethnic_gender = ethnic_gender.unstack()
ethnicity_with_gender = ethnicity_gender.unstack()
ethnicity_with_gender

In [ ]:
#displaying deaths of each gender of White people
white_genders = ethnicity_with_gender['death_yn']['White']
plt.title("Deaths of White Men vs Women")

chart = plt.pie(white_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of Black people
black_genders = ethnicity_with_gender['death_yn']['Black']
plt.title("Deaths of Black Men and Women")
chart = plt.pie(black_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of Hispanic people
hispanic_genders = ethnicity_with_gender['death_yn']['Hispanic']
plt.title("Deaths of Hispanic Men and Women")
chart = plt.pie(hispanic_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of Asian people
asian_genders = ethnicity_with_gender['death_yn']['Asian']
plt.title("Deaths of Asian Men and Women")
chart = plt.pie(asian_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of Native American people
native_genders = ethnicity_with_gender['death_yn']['Native']
plt.title("Deaths of Native American Men and Women")
chart = plt.pie(native_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of the other races
other_genders = ethnicity_with_gender['death_yn']['Other']
plt.title("Deaths of Other Men and Women")
chart = plt.pie(other_genders,autopct='%.2f',labels=("Women","Men"))

In [ ]:
#displaying deaths of each gender of Pacific Islander people
pac_genders = ethnicity_with_gender['death_yn']['Pacific Islander']
plt.title("Deaths of Pacific Islander Men and Women")
chart = plt.pie(pac_genders,autopct='%.2f',labels=("Women","Men"))

### Some interesting things to note here.
White people and black people are pretty similar, with women being 60% more likely to survive than men. Pacific Islanders and hispanic people have vastly differing numbers, with women surviving around 70% more than men. This may be related to the fact that a higher percentage of women stay home than men, who tend to be more employed(1). This could also be cultural, especially the difference with Asian women vs men, as only 6.5% of Asian women are employed as of 2019(2).

1: https://work.chron.com/male-vs-female-statistics-workplace-america-23880.html

2: https://www.bls.gov/cps/cpsaat11.htm

In [ ]:
#building dataframe for race with co-morbidity
#building new dataframe for co-morbidity and race
med = file[['medcond_yn','race_ethnicity_combined',"death_yn"]]
#getting the deaths of different races
ethnicity_med = med.groupby([med['medcond_yn'],med['race_ethnicity_combined']]).sum()
#getting the total cases of different races
ethnic_med = med.groupby([med['medcond_yn'],med['race_ethnicity_combined']]).count()
ethnic_med = ethnic_med.unstack()
ethnicity_with_med = ethnicity_med.unstack()
ethnic_med

In [ ]:
#these are the deaths of each race and
#whether or not they had pre-existing conditions
ethnicity_with_med

In [ ]:
#survival rate of Pacific Islanders with
#a pre-existing medical condition
pac_med = ethnicity_with_med['death_yn']['Pacific Islander']
plt.title("Deaths of Pacific Islanders with a Pre-existing Condition")
chart = plt.pie(pac_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of Black people with
#a pre-existing medical condition
black_med = ethnicity_with_med['death_yn']['Black']
plt.title("Deaths of Black People with a Pre-existing Condition")
chart = plt.pie(black_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of White people with
#a pre-existing medical condition
white_med = ethnicity_with_med['death_yn']['White']
plt.title("Deaths of White People with a Pre-existing Condition")
chart = plt.pie(white_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of Hispanic pople with
#a pre-existing medical condition
hispanic_med = ethnicity_with_med['death_yn']['Hispanic']
plt.title("Deaths of Hispanic People with a Pre-existing Condition")
chart = plt.pie(hispanic_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of Asian people with
#a pre-existing medical condition
asian_med = ethnicity_with_med['death_yn']['Asian']
plt.title("Deaths of Asian People with a Pre-existing Condition")
chart = plt.pie(asian_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of Native Americans with
#a pre-existing medical condition
native_med = ethnicity_with_med['death_yn']['Native']
plt.title("Deaths of Native Americans with a Pre-existing Condition")
chart = plt.pie(native_med,autopct='%.2f',labels=("Survivors","Deaths"))

In [ ]:
#survival rate of other people with
#a pre-existing medical condition
other_med = ethnicity_with_med['death_yn']['Other']
plt.title("Deaths of Other Races with a Pre-existing Condition")
chart = plt.pie(other_med,autopct='%.2f',labels=("Survivors","Deaths"))

## Here we see that pre-existing conditions is a MAJOR factor in survivability
It's interesting to note that Pacific Islanders with pre-existing conditions have such a higher survival rate than all of the other ethnic groups at 18.92.
Unfortunaly, we again see that black people have been hit hard by covid, with only 3.7% survivability with a pre-existing condition.
Everybody else has a survivability of around 6-7% with a pre-existing condition.

## Date of Contraction

In [ ]:
#grouping the dataframe to compare time of first positive specimen with the age group
dates = file[['month','death_yn','age_group']]

#getting number of deaths
deaths = dates.groupby([dates['month'],dates['age_group']]).sum()
deaths = deaths.unstack()
deaths.head()

In [ ]:
#graphing the deaths of each agegroup for each month
deaths.plot.bar(stacked=True, figsize = (15,8),
                title="Deaths of Different Age Groups Over Time",)

### Here we can see that older people are dying with much more frequency than younger people, and that the older a patient is, the more likely they are to die.

In [ ]:
#counting the total reported cases in each age group for each month
total_cases = dates.groupby([dates['month'],dates['age_group']]).count()
total_cases = total_cases.unstack()
total_cases = total_cases.rename(columns={"death_yn":'cases'})
total_cases

In [ ]:
#graphing total cases
total_cases.plot.bar(stacked=True, figsize = (15,8),
                    title="Total Cases Over Time")

### Here we can see that younger people are contracting the disease a lot more frequently than older people.
This would likely be due to the social nature of younger people, and the fact that many younger people absolutely need to work to survive. There have been reports(1) about how millenials have much less money saved than other generations had at our age, which would have a prime factor on survivability during lockdown.


!: https://www.businessinsider.com/average-american-millennial-net-worth-student-loan-debt-savings-habits-2019-6#and-the-typical-millennial-has-less-than-5000-in-their-savings-account-3

In [ ]:
#grouping dataframe to compare time of first positive with gender
gender_dates = file[['month','death_yn','sex']]

#getting number of deaths
gender_deaths = gender_dates.groupby([gender_dates['month'],gender_dates['sex']]).sum()
gender_deaths = gender_deaths.unstack()
gender_deaths

In [ ]:
#graphing deaths of each gender by month
gender_deaths.plot.bar(stacked=True, figsize = (15,8),
                      title="Deaths of Different Genders Over Time")

### Here we can see that women tend to survive more often than men, and that over time the balance between the two hasn't really shifted much

In [ ]:
#grouping the dataframe to compare time of first positive specimen with the ethnicity
ethnicity_dates = file[['month','death_yn','race_ethnicity_combined']]

#getting number of deaths
ethnicity_deaths = ethnicity_dates.groupby([ethnicity_dates['month'],ethnicity_dates['race_ethnicity_combined']]).sum()
ethnicity_deaths = ethnicity_deaths.unstack()
ethnicity_deaths

In [ ]:
#graphing the number of deaths per race
ethnicity_deaths.plot.bar(stacked=True, figsize = (15,8),
                         title="Deaths of Different Ethnicities Over Time")

### Here we can see that the distribution of deaths among different racial groups is mostly white, but that's probably because white people are more than 60% percent of the American population.

In [ ]:
#grouping the dataframe to compare time of first positive specimen with the ethnicity
ethnicity_dates = file[['month','death_yn','race_ethnicity_combined']]

#getting number of cases
ethnicity_cases = ethnicity_dates.groupby([ethnicity_dates['month'],ethnicity_dates['race_ethnicity_combined']]).count()
ethnicity_cases = ethnicity_cases.unstack()
ethnicity_cases = ethnicity_cases.rename(columns={"death_yn":'cases'})
ethnicity_cases

In [ ]:
#graphing the number of cases per ethnic group
ethnicity_cases.plot.bar(stacked=True, figsize = (15,8),
                        title="Cases of Different Ethnic Groups Over Time")

## Now to see if there's a correlation between hospital occupancy of covid patients and death

In [ ]:
#grouping the dataframe to compare time of first positive specimen with the age group
dates = file[['month','death_yn']]
#dates
#getting number of deaths
deaths = dates.groupby('month').sum()
# deaths = deaths.unstack()
#deaths
occupancy = file[['month', 'percent_of_inpatients_with_covid']]
occupancy = occupancy.groupby('month').mean()
occupancy.index
deaths['percent_of_inpatients_with_covid'] = occupancy['percent_of_inpatients_with_covid']
deaths

In [ ]:
plt.title("Deaths vs Hospital Covid Inpatient Occupancy Percentage")
plt.xlabel("Deaths")
plt.ylabel("Hospital Covid Inpatient Occupancy Percentage")
plt.scatter(deaths['death_yn'],deaths['percent_of_inpatients_with_covid'])

In [ ]:
correlation = st.pearsonr(deaths['death_yn'],deaths['percent_of_inpatients_with_covid'])
correlation

### That's a pretty low correlation coefficient
Being as that is the case there isn't really a strong correlation between hospital occupancy of covid inpatients and death over the year

# Next up, let's look at whether or not the patient was put in ICU, and if that was effective at preventing death over the year

In [ ]:
#grouping the dataframe to compare time of first positive specimen with the age group
dates = file[['month','death_yn']]
#dates
#getting number of deaths
deaths = dates.groupby('month').sum()
# deaths = deaths.unstack()
#deaths
occupancy = file[['month', 'icu_yn']]
occupancy = occupancy.groupby('month').sum()
occupancy.index
deaths['icu_yn'] = occupancy['icu_yn']
deaths

In [ ]:
deaths.plot.bar(title='Deaths Compared to ICU Occupancy Over Time')

### Here we can see that in the early months of the pandemic, there were more deaths than ICU occupants. 
Especially in June, when there was a surge of cases and deaths. Perhaps over time our medical community learned how to better treat covid patients. It's obvious that people were being sent to the ICU but they maybe just didn't know the signs of when a patient needed to go to the ICU. It appears that as time went on, there were not only less deaths, but higher numbers of patients in the ICU than dying, which would suggest that the treatments being recieved in the ICU are working.

## Hospital Capacity

### Is there a relationship between the COVID-19 admittances /hospital in-patient capacity ratio to COVID mortality rates?

In [ ]:
# Read Data file
df = pd.read_csv('./Data/clean_data.csv')

# Display header to get a sense of look and feel
df.head(2)

#Define Month Grouping
grp_stats = df.groupby(['month'])

grp_stats.count().head(2)

In [ ]:
#Calculate Death & Covid inpatients Stats per month

# Sum on Death 
sum_dth = grp_stats["death_yn"].sum()

# Total Population Count
totalcnt = grp_stats["death_yn"].count()

#Average Percent of deaths
pct_dth = sum_dth/totalcnt

#Average Covid inpatient percentage
covid_inpatient_pct = grp_stats["percent_of_inpatients_with_covid"].mean()


#Create a dataframe to enable statistical regression and plotting
grp_stats_df = pd.DataFrame({"In Patient Pct" : covid_inpatient_pct,
                             "Dth_Rate" : pct_dth
                             })
grp_stats_df

In [ ]:
#Set today's date as a variable
date = pd.to_datetime("today").strftime("%m/%d/%Y")

#Calculate correlation coefficient
Correlation_Coef = round(st.pearsonr(grp_stats_df['In Patient Pct'],grp_stats_df['Dth_Rate'])[0],5)
Correlation_Coef

#Calulate linear formula
x_values = grp_stats_df['In Patient Pct']
y_values = grp_stats_df['Dth_Rate']

(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept

#Record results in a dataframe
linear_formula_Df = pd.DataFrame({
    "Corr Analysis":"In Patient Pct vs Dth_Rate",
    "Slope":slope,
    "Intercept":intercept,
    "rValueCorr_Coef":rvalue,
    "PndasCorr_Coef":Correlation_Coef,
    "StdErr":stderr 
} ,index=[0] )
linear_formula_Df

In [ ]:
#Create text for plot overlay
line_equation = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
line_equation

#Plot the linear regression model on top of the scatter plot

# Generate a scatter plot of Max Temp vs. Latitude
# Add the linear regreassion and line to the plot

#Charactorize figure size/layout                       
fig1, ax1 = plt.subplots(figsize=(15, 8))

#Build and format scatter plot based on mean values drawn from the clean_weather_data_df dataframe
plt.scatter(grp_stats_df['In Patient Pct'],grp_stats_df['Dth_Rate'],s=30, edgecolors='black', color="green")

#define title and Lable axis
plt.title('In Patient Pct vs Dth_Rate '+date,fontsize =18)
plt.xlabel('In Patient Pct',fontsize =16)
plt.ylabel('Dth_Rate',fontsize =16)
marker_size = 12

#Establish plot limits
plt.ylim(0, .2)

# Add plot and text of linear regression model 
plt.plot(x_values,regress_values,"r-")
ax1.annotate(line_equation, xy=(.08, .12), xycoords='data',xytext=(0.25, 0.15), textcoords='axes fraction',
             horizontalalignment='right', verticalalignment='top',fontsize=20,color="blue")


#Save image for potential reference in the report
plt.savefig("./Images/Plots/In Patient Pct vs Dth_Rate.png")

plt.show()

We performed a regression analysis using our hospitalization data to compare the COVID-19 admittances/hospital capacity ratio to COVID mortality rates to see if it would provide us an indicator to help guide in targeting the most vulnerable population to receive the vaccination. 
The regression results showed a relatively low correlation coefficient and, as the hospitalization data set otherwise had numerous significant data gaps, we opted not to pursue this further.

### What trends in mortality rates and COVID Admittances were observed by month? 

In [ ]:
# Generate a bar plot showing the inpatient percentage and death rates by month using pandas.

#Plot Chart
measure_plot = grp_stats_df.plot.bar(figsize=(18,8), color= ['green', 'black'],fontsize = 14)
plt.xlabel("Months",fontsize = 16)
plt.ylabel("Rates and Percentages",fontsize = 16)
plt.title("Inpatient Percentage and Death Rates By Month",fontsize = 18)
plt.xticks(rotation=0)


#Save image for potential reference in the report
plt.savefig("./Images/Plots/inpatient_pct_and_dth_rte_plot_by_mnth.png")

plt.show()

We found that early on, there was an extremely high mortality rate versus admissions. A declining mortality rate for the remainder of the year  was observed ending with November reporting 1.5%. 
We noted that even as admissions went up in the latter months (September - November), the mortality rates continued to decline
We attribute the early high percentage rates to the low amount of testing and suggest that the latter months of our data may be more reliable for interpretive use as they appear to be more comprehensive.

In [ ]:
df = pd.read_csv('./Data/clean_data.csv')

# Display header to get a sense of look and feel
df.head(2)

In [ ]:
#Define Month Grouping
grp_stats = df.groupby(['month'])

grp_stats.count().head(2)

In [ ]:
#Calculate Death & Covid inpatients Stats per month

# count on number of cases reported
cnt_reported = grp_stats["cdc_report_dt"].count()

# Total Population Count
totalcnt = len(df)

# Sum on Death 
sum_dth = grp_stats["death_yn"].sum()

#Death Count per Case Count
dth_cnt_per_cs_cnt = sum_dth/cnt_reported

#Create month array for plotting
month_list = df['month'].unique()
month_list.sort()


#Create a dataframe to enable statistical regression and plotting
grp_stats_df = pd.DataFrame({"Case Count Reported" : cnt_reported,
                             "Death Count Reported" : sum_dth,
                             "Death % of Case Count" :dth_cnt_per_cs_cnt,                                                     
                             })
grp_stats_df

### What trending in case occurrence was observed by month?

In [ ]:
# Generate a line plot of death rate per recorded case over time

#Define plot components
x_axis = month_list
dth_pct_case_cnt = grp_stats_df["Death % of Case Count"]
cse_cnt_rptd = grp_stats_df["Case Count Reported"]


#Charactorize figure size/layout                       
fig = plt.figure(figsize=(15, 8))

#Build and format plot 1
ax1 = fig.add_subplot(111)
ax1.plot(x_axis, dth_pct_case_cnt, linewidth=2, markersize=12,marker="o",color="black", label = "Dth_Rte")
plt.xlabel('Months',fontsize =16)
plt.ylabel('Death Rate',fontsize =16)
plt.legend(loc='upper left')

#Build and format plot 2
ax2=ax1.twinx()
ax2.plot(x_axis, cse_cnt_rptd, linewidth=2, markersize=12,marker="o",color="green", label = "Cases")
plt.legend(loc='upper right')
plt.ylabel('Number of Cases',color="green",fontsize =16)

#define title and Lable axis
plt.title('Death Rate versus Recorded Cases Over Time',fontsize =18)

plt.xticks(x_axis)

#Establish plot limits
#plt.ylim(-.02, .3)

#Save image for potential reference in the report
plt.savefig("./Images/Plots/dth_rte_per_recorded_case_over_time", bbox_inches = "tight")

plt.show()

We observed case occurrences with death rate as an overlay.
As with the COVID admissions to mortality plot viewed previously, we found early on, there was an extremely high mortality rate versus admissions. Then a declining mortality rate for the remainder of the year ending in November at 1.5%. 
We noted that even as recorded COVID cases went up, in the latter months (September - November), the mortality rates continued to decline
We attribute the early high percentage rates to the low amount of testing and suggest that the latter months of our data may be more reliable for interpretive use as they appear to be more comprehensive.

In [ ]:
#Define age grouping 11th month only
df_11 = df.loc[df["month"] == 11,:]

grp_age_stats = df_11.groupby(['age_group','month'])

grp_age_stats.count().head(2)

#Age Calculations
# Sum on Death 
sum_age_dth = grp_age_stats["death_yn"].sum()


#Create a dataframe to enable statistical regression and plotting
grp_stats_age_df = pd.DataFrame({"Age at Death" : sum_age_dth                             
                             })
grp_stats_age_df

In [ ]:
# Generate a bar plot showing the Inpatient Cases and Death Rates by Ethnicity in the 11th Month

#Plot Chart

measure_plot = grp_stats_age_df.plot.bar(figsize=(18,8), color= ['black'],fontsize = 14)
plt.xlabel("Age Group",fontsize = 16)
plt.ylabel("Death Count",fontsize = 16)
plt.title("Inpatient Cases and Death Rates by Ethnicity in the 11th Month",fontsize = 18)
plt.xticks(rotation=45)


#Save image for potential reference in the report
plt.savefig("./Images/Plots/dth_cnt_by_age_grp_plot_by_mnth.png")

plt.show()

In [ ]:
#Define Ethnic grouping 11th month only
df_11 = df.loc[df["month"] == 11,:]

grp_ethnic_stats = df_11.groupby(['race_ethnicity_combined','month'])

grp_ethnic_stats.count().head(2)

In [ ]:
#Age Calculations
# Sum on Death 
sum_ethnic_dth = grp_ethnic_stats["death_yn"].sum()

#Count of reported
cnt_ethnic_rptd = grp_ethnic_stats["cdc_report_dt"].count()

#Ratio of death to reported
dth_rate_by_ethnicity = sum_ethnic_dth/cnt_ethnic_rptd

#Create a dataframe to enable statistical regression and plotting
grp_stats_ethnic_df = pd.DataFrame({"Cases Reported":cnt_ethnic_rptd,
                                    "Pct of Reported Dying":dth_rate_by_ethnicity                                    
                             })
grp_stats_ethnic_df

In [ ]:
# Generate a bar plot showing the cases reported and percent of reported dying

#Plot Chart
fig = plt.figure(figsize=(15, 8))

width=0.2

#Build and format plot 1
ax1 = fig.add_subplot(111)
grp_stats_ethnic_df['Pct of Reported Dying'].plot.bar(figsize=(18,8), color= ['black'],fontsize = 14, position=1,width=width)
plt.xlabel('Ethnicity',fontsize =16)
plt.ylabel('Death Rate',fontsize =16)
plt.legend(loc='upper left')
plt.xticks(rotation=45)

#Build and format plot 2
ax2=ax1.twinx()
grp_stats_ethnic_df['Cases Reported'].plot.bar(figsize=(18,8), color= ['green'],fontsize = 14, position=0,width=width)
plt.legend(loc='upper right')
plt.ylabel('Cases Reported',color="green",fontsize =16)
plt.ylim(0, 55000)

#define title and Lable axis
plt.title("Cases Reported and Percent of Reported Dying in the 11th Month",fontsize = 18)



#Save image for potential reference in the report
plt.savefig("./Images/Plots/cases_rptd_and_pct_rptd_dth_by_ethnic_grp.png")

plt.show()

# 3. Regression Analysis

In [ ]:
# Import dataset

df = pd.read_csv('Data/clean_data.csv')
df.head()

## Dummy Variable Classification
Dummy variables are a great way to let the regression software do the heavy lifting in the analysis of causal factors 

The underlying math is actually the same as a groupby average, but it gives our results in terms of regression coefficients, so we can very easily use them to estimate the future

https://conjointly.com/kb/dummy-variables/

In [ ]:
# pandas.get_dummies is custom-built for this

# Age:
ageDummies = pd.get_dummies(df['age_group'])
df = pd.merge(df, ageDummies, left_index=True, right_index=True)

# Ethnicity:
ethnoDummies = pd.get_dummies(df['race_ethnicity_combined'])
df = pd.merge(df, ethnoDummies, left_index=True, right_index= True)

df.head()


In [ ]:
#REGRESSION: LINEAR PROBABILITY MODEL

# Define input variables
x = df[['sex', 'medcond_yn', '0 - 9 Years', '10 - 19 Years', '20 - 29 Years',
       '30 - 39 Years', '40 - 49 Years', '50 - 59 Years', '60 - 69 Years',
       '70 - 79 Years', '80+ Years', 'Native', 'Asian','Black', 'Hispanic',
       'Other','Pacific Islander','White']]
x = sm.add_constant(x)
# Avoid perfect multicollinearity (comment out these two lines and you'll see that it breaks the regression)
del x['0 - 9 Years']
del x['White']

# Define output variable (death)
y = df['death_yn']

# Ordinary Least Squares Regression
model = sm.OLS(y, x, hasconst= True)
res = model.fit()
res.summary()

## Regressing Death: Results
Takeaways:
- Men are 2.73% more likely to die than women (all else equal)
- Those with a medical condition are 3.89% more likely to die than those without (all else equal)
- Age's effect on the likelihood of death is shown in fig. 1
- Ethnicity's effect is shown in fig. 2
 
This is a very promising Linear Probability model. Every single one of our variables has a p-value of 0, meaning our variables are significant. We have quantified many of the risk factors.

In [ ]:
#Figure 1 calculation

plt.figure(figsize = (20,8))
plt.ylabel('Regression Coefficient', fontsize = 15)
plt.xlabel('Age Group', fontsize = 15)
res.params.loc['10 - 19 Years': '80+ Years'].plot(kind = 'bar')
plt.xticks(rotation = 0, fontsize= 15)
plt.title('Fig. 1: Age and Mortality Risk', fontsize = 20)
plt.savefig('Images/Regression/fig1_age.png', facecolor = 'white')
plt.show()


In [ ]:
# Figure 2 calculations

plt.figure(figsize = (20,8))
plt.title('Fig. 2: Ethnicity and mortality risk', fontsize = 20)
plt.ylabel('Regression Coefficient', fontsize = 15)
plt.xlabel('Ethnicity', fontsize = 15)

res.params.loc['Native': 'Pacific Islander'].plot(kind = 'bar', )
plt.xticks(rotation = 0, fontsize= 15)
plt.savefig('Images/Regression/fig2_ethnicity.png', facecolor = 'white')
plt.show()

## Predicting Death
So we have demonstrated strong correlation, but what about predictive power? What do our predicted probabilities of death look like when compared with the actual results?

In [ ]:
# Generate predictions
df['LPM Prediction'] = res.predict(x)

predictions = df[['LPM Prediction', 'death_yn']]

deaths = predictions[predictions['death_yn'] == True]
lives = predictions[predictions['death_yn'] == False]

fig, axs = plt.subplots(1, 2, figsize = (20,8))
fig.suptitle('Fig. 3:\nDistribution of Predictions for the Living and Dead', fontsize= 20)
axs[0].hist(lives['LPM Prediction'])
axs[0].set_title('Survived:', fontsize = 18)
axs[0].set_xlabel('Predicted Probability of Death', fontsize = 15)
axs[0].set_ylabel('Frequency', fontsize = 15)

axs[1].hist(deaths['LPM Prediction'], bins = 10)
axs[1].set_title('Deceased:', fontsize = 18)
axs[1].set_xlabel('Predicted Probability of Death', fontsize= 15)
axs[1].set_ylabel('Frequency', fontsize = 15)

plt.savefig('Images/Regression/fig3_prediction_distribution.png', facecolor = 'white')
plt.show()

## Analyzing Results
It looks like there is a clear pattern between predicted probability of death and actual risk of death. However, the predictions are clearly skewed. The highest predicted probability of death was .46, meaning *no one* was deemed by the model to have a greater chance of dying than surviving. This is due to the fact that only around 6 percent of patients in this dataset ended up dying. 

Even when you look at the population which is most at risk (80+ year olds), 60% of them survived. So it stands to reason that the model would fail to give a patient a 50+% chance of death, when fewer than 50% of even the most vulnerable patients actually passed away.

In [ ]:
# Compare summary statistics for living and dead
dead = pd.DataFrame(df[df['death_yn'] == 1]['LPM Prediction'].describe())
alive = pd.DataFrame(df[df['death_yn'] == 0]['LPM Prediction'].describe())

compare = pd.merge(dead, alive, left_index= True, right_index= True, suffixes = ['_dead', '_alive'])
compare.iloc[1:]

In [ ]:
# Save dataframe for more in-depth regression 
df.to_csv('Data/regression_dataframe.csv', index= False)

# 3.1 Examining the Effects of Medical Conditions
The purpose of this section is to better understand the effect of pre-existing medical conditions on mortality risk from Covid-19. 

When I saw the relatively small coefficient for the medcond_yn variable in the original regression, I thought we probably needed to introduce some interaction variables into the model. In linear regression, an interaction variable allows for the effect of one variable (age, ethnicity) to depend on another variable (medcond_yn). In this case, it is reasonable to assume that the presence of a medical condition will present a different degree of risk for a 20 year-old, a 50-year old, and an 80 year-old.

We can simply multiply the values of our medcond_yn variable with our dummy classification variables for age and ethnicity. The resulting effect is two regression coefficients per age / ethnicity group. One for those with, and one for those without a pre-existing medical condition. For further reading, check out https://www.theanalysisfactor.com/interaction-dummy-variables-in-linear-regression/

In [ ]:
df = pd.read_csv('Data/regression_dataframe.csv')
df.head()

In [ ]:
deaths = df.groupby('medcond_yn').sum()['death_yn']
counts = df.groupby('medcond_yn').count()['death_yn']
frame = pd.DataFrame({'Deaths': deaths, 'Cases': counts})
frame['Pct'] = 100 * (frame['Deaths'] / frame['Cases'])
frame.index =['Without', 'With']
frame['Pct'].plot(kind='bar', figsize=(20,8))
plt.ylabel('Percent who died', fontsize = 15)
plt.tick_params(labelrotation = 0)
plt.xticks( fontsize = 15)
plt.title('Fig. 4:\nOverall Mortality Rates for Those With and Without Medical Conditions', fontsize = 20)
plt.savefig('Images/Regression/fig4_medcond_distribution.png', facecolor = 'white')
frame

In [ ]:
# Declare which columns will be interacted with
columns = ['20 - 29 Years', '30 - 39 Years',
       '40 - 49 Years', '50 - 59 Years', '60 - 69 Years', '70 - 79 Years',
       '80+ Years', 'Asian', 'Black', 'Hispanic', 'Native', 'Other',
       'Pacific Islander']

interactions = []
# Multiply each column with medcond_yn to create a new column
for col in columns:
    colName = col + '_interact'
    df[colName] = df[col] * df['medcond_yn']
    interactions.append(colName)
df[interactions].head()


In [ ]:
# Perform analysis
exog = ['sex'] + columns + interactions + ['medcond_yn']
x = df[exog]
x = sm.add_constant(x)

y = df['death_yn']

model = sm.OLS(y, x, hasconst = True)
res = model.fit()
res.summary()

# Interpretation
Woah! Big change in coefficients!

In the first regression, we estimated that being 80+ years old increased an individual's likelihood of death by around 40%. Now that we have distinguished between those with a medical condition and those without, that number has dropped to 20%. However, 80 year olds *with* a medical condition have an estimated mortality risk of *over* 40% (20.75 + 23.54). 

Where we before estimated that 60-69 year-old patients had an almost 10% increased risk of death, we now have them at just 2%! However, those with a medical condition have around a 14% (2.23 + 11.46) risk of dying!

The inclusion of this interaction variable has really enhanced the specificity of our results. Let's visualize each age group's mortality, ***taking pre-existing conditions into consideration*** 

In [ ]:
# Plot Setup:
fig, axs = plt.subplots(1, 2, figsize = (25,10))
fig.suptitle('Fig. 5:\nMedical Conditions and Age', fontsize = 20)

axs[0].set_ylabel('Mortality Risk', fontsize = 15)
axs[1].set_ylabel('Pct of Group Deceased', fontsize = 15)
axs[0].set_title('Regression Coefficients', fontsize = 20)
axs[1].set_title('Actual Mortality', fontsize = 20)


# Coefficients:
Without = res.params.loc['20 - 29 Years': '80+ Years']
With = res.params.loc['20 - 29 Years_interact':'80+ Years_interact']

coefficients = pd.DataFrame({'Without': Without.values, 'With': Without.values + With.values})
coefficients.set_index(Without.index, inplace = True)

coefficients.plot(kind='bar', ax = axs[0])
axs[0].tick_params(labelrotation = 0)
axs[0].set_xlabel('Age Group', fontsize = 15)
axs[0].legend(['Without', 'With'], fontsize = 20)
#Actual Values:
# Groupby age group and medical condition - compare share of death
sums = df.groupby(['age_group', 'medcond_yn']).sum()['death_yn']
counts = df.groupby(['age_group', 'medcond_yn']).count()['death_yn']
compare = pd.merge(sums, counts, left_index = True, right_index = True).rename(columns = {'death_yn_x': 'Total Deaths',
                                                                                         'death_yn_y': 'Total Cases'})
compare['Pct Dead'] = (compare['Total Deaths'] / compare['Total Cases']) * 100
compare = compare['Pct Dead'].unstack().rename(columns= {0: 'Without', 1: 'With'})

compare.plot(kind='bar', ax = axs[1])
axs[1].set_xlabel('Age Group', fontsize = 15)
axs[1].tick_params(labelrotation = 0)
axs[1].legend(['Without', 'With'], fontsize = 20)

plt.savefig('Images/Regression/fig5_age_and_medcond.png', facecolor = 'white')
plt.show()

In [ ]:
# Plot Setup:
fig, axs = plt.subplots(1, 2, figsize = (25,10))
fig.suptitle('Fig. 6:\nMedical Conditions and Ethnicity', fontsize = 20)

axs[0].set_ylabel('Mortality Risk', fontsize = 15)
axs[1].set_ylabel('Pct of Group Deceased', fontsize = 15)
axs[0].set_title('Regression Coefficients', fontsize = 20)
axs[1].set_title('Actual Mortality', fontsize = 20)


# Coefficients:
Without = res.params.loc['Asian': 'Pacific Islander']
With = res.params.loc['Asian_interact':'Pacific Islander_interact']

coefficients = pd.DataFrame({'Without': Without.values, 'With': Without.values + With.values})
coefficients.set_index(Without.index, inplace = True)

coefficients.plot(kind='bar', ax = axs[0])
axs[0].tick_params(labelrotation = 0)
axs[0].set_xlabel('Age Group', fontsize = 15)
axs[0].legend(['Without', 'With'], fontsize = 20)
#Actual Values:
# Groupby age group and medical condition - compare share of death
sums = df.groupby(['race_ethnicity_combined', 'medcond_yn']).sum()['death_yn']
counts = df.groupby(['race_ethnicity_combined', 'medcond_yn']).count()['death_yn']
compare = pd.merge(sums, counts, left_index = True, right_index = True).rename(columns = {'death_yn_x': 'Total Deaths',
                                                                                         'death_yn_y': 'Total Cases'})
compare['Pct Dead'] = (compare['Total Deaths'] / compare['Total Cases']) * 100
compare = compare['Pct Dead'].unstack().rename(columns= {0: 'Without', 1: 'With'})

compare.plot(kind='bar', ax = axs[1])
axs[1].set_xlabel('Age Group', fontsize = 15)
axs[1].tick_params(labelrotation = 0)
axs[1].legend(['Without', 'With'], fontsize = 20)

plt.savefig('Images/Regression/fig6_ethnicity_and_medcond.png', facecolor = 'white')
plt.show()

## Interpretation
So this graph tells  us two things:
1. Asian and Black people see the highest increased risk of mortality with the presence of pre-existing conditions
2. The presence of pre-existing medical conditions is a much higher risk factor than what the first model initally suggested 

## Why the big change?
Recall that in the first model, the presence of a medical condition was estimated to increase mortality risk by just 3 percent. When specifying by ethnicity and age, however, we see much larger coefficients for the presence of a medical condition. Before, we had not properly specified the model somehow. But how?

One possibility is that the majority of those with medical conditions are young, and they see very little increased risk. If the majority of patients in this dataset are young, then an unspecified regression analysis would see that the majority of those with medical conditions do **not** pass away.

Another possibility is that the majority of those with medical conditions are very *old*, and the regression has "confused" the effect of age with the effect of medical condition. If the majority of those with medical conditions are 80+ years old, and 80+ year-olds see the highest rate of mortality, then an unspecified regression analysis would see a much stronger correlation between *age* and mortality than between the presence of medical conditions and mortality.

Lets look at the distribution of medical conditions by age and see if either of these guesses are correct.

In [ ]:
# Group by age group and show intra-group frequency of medical conditions
sums = df.groupby('age_group').sum()['medcond_yn']
counts = df.groupby('age_group').count()['medcond_yn']

totals = pd.DataFrame({'Total With Conditions': sums, 'Total': counts})
totals['Pct'] = totals['Total With Conditions'] / totals['Total']
totals['Pct'].plot(kind = 'bar', figsize = (20,8))
totals['Pct of Pop'] = totals['Total'] / df.shape[0]
plt.ylabel('Pct of Age Group with a Condition', fontsize = 15)
plt.xlabel('Age Group', fontsize = 15)
plt.xticks(rotation = 0, fontsize = 13)
plt.yticks(fontsize = 13)
plt.title('Fig. 7:\nAge Groups and the Frequency of Medical Conditions', fontsize = 20)
plt.savefig('Images/Regression/fig7_age_medcond_distribution.png', facecolor = 'white')
plt.show()

## Age Bias
It does, in fact, look like the presence of medical conditions is heavily skewed with age. Not only that, but the majority of observations in this dataset are of individuals younger than 60 years old. In other words, most observations are of people for whom the presence of a medical condition increases mortality risk by less than 10%

This explains why we see such a huge disparity in the medcond_yn coefficients before and after including the interaction variables.

By specifying the intra-group presence of medical conditions, we have painted a much clearer picture of the causes of mortality 

## Predictive Power
Now that we have re-specified our model, we can check its predictive power again.

In [ ]:
# Replace current prediction estimates (from the last regression)
df['LPM Prediction'] = res.predict(x)
deaths = df[df['death_yn'] == 1]
lives = df[df['death_yn'] == 0]

fig, axs = plt.subplots(1, 2, figsize = (20,8))
fig.suptitle('Fig. 8:\nImproved Distribution of Predictions for the Living and Dead', fontsize = 20)
lives['LPM Prediction'].hist(ax = axs[0])
deaths['LPM Prediction'].hist(ax = axs[1])
for ax in axs:
    ax.set_xlabel('Predicted Death Risk', fontsize = 15)
    ax.set_ylabel('# of Occurences', fontsize = 15)

axs[0].set_title('Survived', fontsize = 15)
axs[1].set_title('Deceased', fontsize = 15)

plt.savefig('Images/Regression/fig8_improved_prediction_distribution.png')
plt.show()

## Better fit
Our prediction distribution looks **very** similar to the one from the first regression (Fig. 3) However, it is more skewed in the correct directions (left for the survived, right for the deceased). We have improved our model!
# 3.2 Changes over time
Thus far, we have not taken into account that the treatment options for Covid-19 have greatly improved over time. If we isolate the effect of time's passing, it will better enable us to identify the causal factors behind mortality risk.

In [ ]:
# First, let's visualize it
dead = df.groupby('month').sum()['death_yn']
counts = df.groupby('month').count()['death_yn']
deaths_by_month = pd.DataFrame({'Deaths': dead, 'Cases': counts})
deaths_by_month['Pct'] = (deaths_by_month['Deaths'] / deaths_by_month['Cases']) * 100
deaths_by_month['Pct'].plot(figsize = (20,8), marker = 'o', markerfacecolor = 'r')
plt.xlabel('Month', fontsize = 15)
plt.ylabel('Percent of patients who died', fontsize = 15)
plt.title('Fig. 9:\nOverall Death Rate Over Time', fontsize = 20)
plt.savefig('Images/Regression/fig9_death_over_time.png')

In [ ]:
# Grab the occupancy column from the main dataframe
cap = pd.read_csv('Data/clean_data.csv')['inpatient__occupancy']
df['Hospital Capacity'] = cap
cap_by_month = df.groupby('month').mean()['Hospital Capacity'] * 100

# We got the column. Now let's plot it
deaths_by_month['Pct'].plot(figsize = (20,8), marker = 'o', markerfacecolor = 'r')
cap_by_month.plot()
plt.xlabel('Month', fontsize = 15)
plt.ylabel('Percent of patients who died', fontsize = 15)
plt.title('Fig. 10:\nDeath Rate and Hospital Capacity', fontsize = 20)
plt.legend(['Death Rate', 'Capacity'], fontsize = 15)
plt.savefig('Images/Regression/fig10_death_and_capacity.png')

It seems that over time, hospital occupancy has gone up, and the death rate has gone down in spite of this fact. Now, let's quantify the time effect by introducing some time-fixed-effects into our model. Not only will this reflect the trend seen above in Fig. 10, but it will allow the regression software to specify its other coefficients even more accurately due to the mitigation of ommitted variable bias.

In [ ]:
months = pd.get_dummies(df['month'])
months

# This for loop is the same as a merge. 
for i in range(2, 12):
    x['month ' + str(i)] = months[i]

x.head()

In [ ]:
model = sm.OLS(y, x, hasconst = True)
res = model.fit()
res.summary()

At first glance, it looks like the size of most of our coefficients have remained the same

In [ ]:
res.params.loc['month 2': 'month 11'].plot(kind = 'bar', figsize = (20,8))
plt.tick_params(labelrotation = 0)
plt.xlabel('Month', fontsize = 15)
plt.ylabel('Coefficient', fontsize = 15)
plt.title('Fig. 11:\nTime Fixed Effects', fontsize = 20)
plt.savefig('Images/Regression/fig11_time_fixed_effects')

## Model Respecification
By introducing these time-fixed-effects, we have certainly changed the regression coefficients. Let's see if our above observations continue to hold water after re-specifying the model 

***Note***: *The below code is a copy-paste from the code that generated figures 5 and 6*

In [ ]:
# Plot Setup:
fig, axs = plt.subplots(1, 2, figsize = (25,10))
fig.suptitle('Fig. 12:\nMedical Conditions and Age\n(After Inclusion of Fixed Effects)', fontsize = 20)

axs[0].set_ylabel('Mortality Risk', fontsize = 15)
axs[1].set_ylabel('Pct of Group Deceased', fontsize = 15)
axs[0].set_title('Regression Coefficients', fontsize = 20)
axs[1].set_title('Actual Mortality', fontsize = 20)


# Coefficients:
Without = res.params.loc['20 - 29 Years': '80+ Years']
With = res.params.loc['20 - 29 Years_interact':'80+ Years_interact']

coefficients = pd.DataFrame({'Without': Without.values, 'With': Without.values + With.values})
coefficients.set_index(Without.index, inplace = True)

coefficients.plot(kind='bar', ax = axs[0])
axs[0].tick_params(labelrotation = 0)
axs[0].set_xlabel('Age Group', fontsize = 15)
axs[0].legend(['Without', 'With'], fontsize = 20)
#Actual Values:
# Groupby age group and medical condition - compare share of death
sums = df.groupby(['age_group', 'medcond_yn']).sum()['death_yn']
counts = df.groupby(['age_group', 'medcond_yn']).count()['death_yn']
compare = pd.merge(sums, counts, left_index = True, right_index = True).rename(columns = {'death_yn_x': 'Total Deaths',
                                                                                         'death_yn_y': 'Total Cases'})
compare['Pct Dead'] = (compare['Total Deaths'] / compare['Total Cases']) * 100
compare = compare['Pct Dead'].unstack().rename(columns= {0: 'Without', 1: 'With'})

compare.plot(kind='bar', ax = axs[1])
axs[1].set_xlabel('Age Group', fontsize = 15)
axs[1].tick_params(labelrotation = 0)
axs[1].legend(['Without', 'With'], fontsize = 20)

plt.savefig('Images/Regression/fig12_age_and_medcond_post_FE.png', facecolor = 'white')
plt.show()

In [ ]:
# Plot Setup:
fig, axs = plt.subplots(1, 2, figsize = (25,10))
fig.suptitle('Fig. 13:\nMedical Conditions and Ethnicity\n(After Inclusion of Fixed Effects)', fontsize = 20)

axs[0].set_ylabel('Mortality Risk', fontsize = 15)
axs[1].set_ylabel('Pct of Group Deceased', fontsize = 15)
axs[0].set_title('Regression Coefficients', fontsize = 20)
axs[1].set_title('Actual Mortality', fontsize = 20)


# Coefficients:
Without = res.params.loc['Asian': 'Pacific Islander']
With = res.params.loc['Asian_interact':'Pacific Islander_interact']

coefficients = pd.DataFrame({'Without': Without.values, 'With': Without.values + With.values})
coefficients.set_index(Without.index, inplace = True)

coefficients.plot(kind='bar', ax = axs[0])
axs[0].tick_params(labelrotation = 0)
axs[0].set_xlabel('Age Group', fontsize = 15)
axs[0].legend(['Without', 'With'], fontsize = 20)
#Actual Values:
# Groupby age group and medical condition - compare share of death
sums = df.groupby(['race_ethnicity_combined', 'medcond_yn']).sum()['death_yn']
counts = df.groupby(['race_ethnicity_combined', 'medcond_yn']).count()['death_yn']
compare = pd.merge(sums, counts, left_index = True, right_index = True).rename(columns = {'death_yn_x': 'Total Deaths',
                                                                                         'death_yn_y': 'Total Cases'})
compare['Pct Dead'] = (compare['Total Deaths'] / compare['Total Cases']) * 100
compare = compare['Pct Dead'].unstack().rename(columns= {0: 'Without', 1: 'With'})

compare.plot(kind='bar', ax = axs[1])
axs[1].set_xlabel('Age Group', fontsize = 15)
axs[1].tick_params(labelrotation = 0)
axs[1].legend(['Without', 'With'], fontsize = 20)

plt.savefig('Images/Regression/fig13_ethnicity_and_medcond_post_FE.png', facecolor = 'white')
plt.show()

One noticeable change: There are now quite a few negative coefficients. If you look close, the pattern is the same as before. 

In other words, ***Relative to each other***, all of our coefficients have maintained the same values. We have simply given more accuracy to the model **overall** by introducing these time-fixed-effects. Our R-squared value rose by 15 percent (0.212 --> 0.244), so that is a good sign. An even better picture would be to repeat the excercise where we show the distribution of predicted probabilities.

*Note: R-squared went up, and that is a positive sign, but for Linear Probability Models, R-squared is a very poor measure of quality. It cannot be interpreted intuitively. That it rose is a good sign, but its value should always be taken with a grain of salt, especially in the case of a Linear Probability Model*

In [ ]:
# Replace current prediction estimates (from the last regression)
df['LPM Prediction'] = res.predict(x)
deaths = df[df['death_yn'] == 1]
lives = df[df['death_yn'] == 0]

fig, axs = plt.subplots(1, 2, figsize = (20,8))
fig.suptitle('Fig. 14:\nImproved Distribution of Predictions for the Living and Dead', fontsize = 20)
lives['LPM Prediction'].hist(ax = axs[0])
deaths['LPM Prediction'].hist(ax = axs[1])
for ax in axs:
    ax.set_xlabel('Predicted Death Risk', fontsize = 15)
    ax.set_ylabel('# of Occurences', fontsize = 15)

axs[0].set_title('Survived', fontsize = 15)
axs[1].set_title('Deceased', fontsize = 15)

plt.savefig('Images/Regression/fig14_improved_prediction_distribution_post_FE.png')
plt.show()

The distributions have moved *even further* in the correct directions.